<img style="float: left;" src="figures/model2.png" width="20%">   

# <font color='Red'> SPE11 benchmark </font>

## <font color='Blue'>Introduction</font>

The SPE11b model is the 2D reservoir model of the 11th Society of Petroleum Engineers (SPE) comparative solution project (CSP). The CSP provides a baseline for modeling of geological sequestration of CO$_2$. In this jupyter notebook you will run the SPE11b model in DARTS.

## <font color='blue'>The objectives:</font>
We use predefined python model using the following files: 
 * File [SPE11 geometry](https://gitlab.com/open-darts/darts-models/-/blob/development/teaching/CCS_workshop/SPE11/fluidflower.py) with main converters and plotters
 * File [SPE11b discretizer](https://gitlab.com/open-darts/darts-models/-/blob/development/teaching/CCS_workshop/SPE11/fluidflower_str_b.py) based on structured reservoir
 * File [SPE11 model](https://gitlab.com/open-darts/darts-models/-/blob/development/teaching/CCS_workshop/SPE11/model_b.py) with physics and properties

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from SPE11.model_b import Model, PorPerm, Corey, layer_props
from darts.engines import redirect_darts_output, sim_params
from SPE11.fluidflower_str_b import FluidFlowerStruct

## <font color='Blue'>Define the physics and parameters</font>

In the physics class of the `DartsModel()` all the thermodynamic and phase properties of the model are defined. These are assembled with the `PropertyContainer()` class and subsequently accessed by the `Physics()` class to compute supporting points for interpolation. Here we use the `Compositional()` class, that uses a generalized thermo-compsitional formulation that can be tailored to DeadOil, Compositional and CCS applications. To control the number of OBL supporting points change the parameter `n_points`. 

<img style="float: left;" src="figures/caseb.png" width="90%">   

In [ ]:
# For each of the facies within the SPE11b model we define a set of operators in the physics.
property_regions  = [0, 1, 2, 3, 4, 5, 6] 
layers_to_regions = {"1": 0, "2": 1, "3": 2, "4": 3, "5": 4, "6": 5, "7": 6}

# define the Corey parameters for each layer (rock type) according to the technical description of the CSP
corey = {
    0: Corey(nw=1.5, ng=1.5, swc=0.32, sgc=0.10, krwe=1.0, krge=1.0, labda=2., p_entry=1.935314, pcmax=300, c2=1.5),
    1: Corey(nw=1.5, ng=1.5, swc=0.14, sgc=0.10, krwe=1.0, krge=1.0, labda=2., p_entry=0.08655, pcmax=300, c2=1.5),
    2: Corey(nw=1.5, ng=1.5, swc=0.12, sgc=0.10, krwe=1.0, krge=1.0, labda=2., p_entry=0.0612, pcmax=300, c2=1.5),
    3: Corey(nw=1.5, ng=1.5, swc=0.12, sgc=0.10, krwe=1.0, krge=1.0, labda=2., p_entry=0.038706, pcmax=300, c2=1.5),
    4: Corey(nw=1.5, ng=1.5, swc=0.12, sgc=0.10, krwe=1.0, krge=1.0, labda=2., p_entry=0.0306, pcmax=300, c2=1.5),
    5: Corey(nw=1.5, ng=1.5, swc=0.10, sgc=0.10, krwe=1.0, krge=1.0, labda=2., p_entry=0.025602, pcmax=300, c2=1.5),
    6: Corey(nw=1.5, ng=1.5, swc=1e-8, sgc=0.10, krwe=1.0, krge=1.0, labda=2., p_entry=1e-2, pcmax=300, c2=1.5)
}

redirect_darts_output('model2.log') # redirects run.log to your directory of choice instead of prininting everything off

"""Define realization ID"""
model_specs = [
    {'structured': True, 
     'thickness': False, 
     'curvature': False, 
     'tpfa': True, 
     'capillary': True, 
     'nx': 170, 
     'nz': 60, 
     'output_dir': 'SPE11_output'},
]

j = 0
specs = model_specs[j]  
m = Model() 
m.temperature = 323.15

"""Define physics"""
zero = 1e-10
m.set_physics(corey=corey, zero=zero, n_points=1001, diff=1e-8)

# solver paramters 
m.set_sim_params(first_ts=1e-2, mult_ts=2, max_ts=365, tol_linear=1e-3, tol_newton=1e-3,
                 it_linear=50, it_newton=12)
m.params.nonlinear_norm_type = m.params.L1

## <font color='Blue'>Define the reservoir</font>

The 11b model is a reservoir model at reservoir conditions. The horizontal and vertical extent of the model are equal to 8400m and 1200m. To start the the total number of grid blocks is limited to 10K. Feel free to play with the nx and nz paramters and change the resolution of the model. Within the reservoir we can clearly identify two low permeability layers that will function as capillary barriers in the storage reservoir.

In [ ]:
"""Define the reservoir and wells """
well_centers = {
    "I1": [2700.0, 0.0, 300.0],
    "I2": [5100.0, 0.0, 700.0]
}

structured = specs['structured']
m.reservoir = FluidFlowerStruct(timer=m.timer, layer_properties=layer_props, layers_to_regions=layers_to_regions,
                                model_specs=specs, well_centers=well_centers) # structured reservoir 


grid = np.meshgrid(np.linspace(0, 8400, m.reservoir.nx), np.linspace(0, 1200, m.reservoir.nz))

plt.figure(figsize = (10, 2))
plt.title('Porosity')
c = plt.pcolor(grid[0], grid[1], m.reservoir.global_data['poro'].reshape(m.reservoir.nz, m.reservoir.nx))
plt.colorbar(c)
plt.xlabel('x [m]') 
plt.ylabel('z [m]')
plt.show()

## <font color='Blue'>Validate properties</font>

Properties can be evaluated via the property container. In this example xCO2 (dissolved CO2 in Aqueous phase) and aqueous phase density are evaluated for different pressures at the injection temperature 10C. Change the temperatue and see how it effects density and solubility of CO2 in the aqueous phase. Note xi denotes the mole fraction of component i in liquid and yi the mole fraction of component i in gas. 

In [ ]:
facies = 0
property_container = m.physics.property_containers[facies]
component_names = property_container.components_name
phase_names = property_container.phases_name

temperature = [10, 50,  100] # celsius
pressure = np.linspace(200, 450, 10) # bar
zc = 0.15

x = np.zeros(pressure.shape)
dens = np.zeros(pressure.shape)
phase = 1 # phase 
component = 1 # component

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
for t in temperature:
    for i, p in enumerate(pressure): 
        property_container.run_flash(p, 273.15+t, [zc, 1-zc])
        x[i] = property_container.x[phase, component]
        dens[i] = property_container.density_ev['Aq'].evaluate(p, 273.15+t, property_container.x[phase, :])
        
    ax1.grid()
    ax1.plot(pressure, x, label=f"{t} C")
    ax1.set_xlabel("Pressure (bar)")
    ax1.set_ylabel(f"x{component_names[component]}" if phase == 1 else f"y{component_names[component]}" )
    ax1.legend()
    ax2.grid()
    ax2.plot(pressure, dens, label=f"{t} C")
    ax2.set_xlabel("Pressure (bar)")
    ax2.set_ylabel(f"Density {phase_names[phase]} (kg/m³)")
    ax2.legend()
    plt.subplots_adjust(wspace = 0.4)
plt.show()

## <font color='Blue'>Initialize the model</font>

In [ ]:
""" Define initial and boundary conditions """
# define initial pressure, composition and temperature of the reservoir
pres_in = 212
pres_grad = 0.09775
m.input_depth = [0., 1000.]
m.input_distribution = {"pressure": [pres_in, pres_in + pres_grad * m.input_depth[1]],
                        "H2O": 1. - zero,
                        "temperature": [m.temperature, m.temperature]}

# define injection stream of the wells 
m.inj_stream = [zero] 

inj_rate = 3024 # mass rate per well, kg/day
m.inj_rate = [0, 0] # per well

m.set_str_boundary_volume_multiplier()  # right and left boundary volume multiplier

# now that your reservoir and physics is defined, you can init your DartsModel()
output_dir = specs['output_dir']
m.platform = 'cpu' 
m.init(discr_type='tpfa', platform=m.platform)
m.set_output(output_folder = 'output/SPE11', save_initial=False, verbose=True)

# equillibration step 
m.run(365, save_well_data=False, verbose = False) 
m.physics.engine.t = 0 # return engine time to zero

## <font color='Blue'>Run the model</font>

The model is run for 1000 years. Injection in well 1 (bottom well) starts in year 0 and continues untill year 50. In year 25 injection at well 2 (top well) starts and ends in year 50.

In [ ]:
m.inj_rate = [inj_rate, 0] # [well 1, well 2]
Nt = 20 # 100 years with output every 5 years
import time 

start = time.time()
for i in range(Nt):
    m.run(5*365, save_well_data=False, verbose = True) # run model for 1 year
    
    if m.physics.engine.t >= 25 * 365 and m.physics.engine.t < 50 * 365:
        # At 25 years, start injecting in the second well
        m.inj_rate = [inj_rate, inj_rate]  
    
    elif m.physics.engine.t >= 50 * 365:
        # At 50 years, stop injection for both wells
        m.inj_rate = [0, 0]    
stop = time.time()
print("Runtime = %3.2f sec" % (stop - start))

## <font color='Blue'>Postprocess solution</font>

In [ ]:
nx, nz = m.reservoir.nx, m.reservoir.nz  # grid dimensions
M_H2O = m.physics.property_containers[0].Mw[0] # molar mass water in kg/kmol 
M_CO2 = m.physics.property_containers[0].Mw[1] # molar mass CO2 in kg/kmol
PV = np.array(m.reservoir.mesh.volume)[1] * np.array(m.reservoir.mesh.poro) # pore volume 

# Generate some example data to animate
prop_list = m.physics.vars + m.output.properties; print(prop_list)

# evaluate properties at all the saved time steps, according to the listed properties in prop_list
time_vector, property_array = m.output.output_properties(output_properties = prop_list, timestep = None) 

mass_co2 = np.zeros((len(time_vector), nx*nz))
mass_co2_v = np.zeros((len(time_vector), nx*nz))
mass_co2_aq = np.zeros((len(time_vector), nx*nz))
for i in range(len(time_vector)):    
    # vapour mass fraction 
    wco2 = property_array['yCO2'][i] * M_CO2 / (property_array['yCO2'][i] * M_CO2 + (1 - property_array['yCO2'][i]) * M_H2O) 

    # mass of CO2 in the aqueous phase 
    mass_co2_aq[i,:] = PV * (1 - property_array['satV'][i]) * property_array['xCO2'][i] * property_array['rho_mA'][i] * M_CO2

    # mass of CO2 in the vapor phase 
    mass_co2_v[i,:] = PV * property_array['satV'][i] * property_array['rhoV'][i] * wco2

    # total mass of CO2 in kton 
    mass_co2[i,:] = (mass_co2_aq[i,:] + mass_co2_v[i,:])/1e6

# since the property array is a dictionary it can easily be appended with the calculated values
property_array['mass_co2'] = mass_co2
property_array['mass_co2_v'] = mass_co2_v
property_array['mass_co2_aq'] = mass_co2_aq
print(property_array.keys())
m.output.save_property_array(time_vector, property_array) # and the property_array can be saved as a compressed HDF5 file in m.output_folder 

## <font color='Blue'>Plot animation</font>

In [ ]:
from matplotlib.animation import FuncAnimation

data = [property_array['mass_co2'][i].reshape(nz, nx) for i in range(len(time_vector))]

# Initialize the figure and color plot
fig, ax = plt.subplots(figsize=(16, 4))
pcolor_plot = ax.pcolor(grid[0], grid[1], data[0], vmin = data[1].min(), vmax = 0.2, cmap = 'jet')
plt.colorbar(pcolor_plot, ax=ax)
ax.set_xlabel("x [m]")
ax.set_ylabel("z [m]")

# Define the update function for animation
def update(frame):
    ax.set_title(f"mCO2 @ year {frame*5}")
    pcolor_plot.set_array(data[frame].ravel())  # Update the data
    return pcolor_plot,

# Create the animation
ani = FuncAnimation(fig, update, frames=len(time_vector), blit=True)

# Display the animation in the Jupyter Notebook
from IPython.display import HTML
HTML(ani.to_jshtml())

## <font color='Blue'>Tasks in this workshop (check and explain why the solution behaves this way):</font>

1. Run the simulation up to 1000 years.
2. Increase the temperature by 50 degrees.
3. Check what happened if diffusion is 1 order higher.

More details on the SPE11 benchmark can be found at https://www.spe.org/en/csp/.